In [2]:


import pandas as pd
import os

import rdkit 

from schnetpack.datasets import QM9

from DLChem.utils import utils

from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem.Fingerprints import FingerprintMols


from pandas import DataFrame

from numpy import savetxt

In [5]:

#label file
element = 'O'
name_label = '5000'
label_file = '../../data/label_qm9/O/label%s%s.csv' %(element,name_label)

#data file
qm9data = QM9('../../data/QM9/qm9.db', download=False, remove_uncharacterized=True)

#number_molecules = 2
label = pd.read_csv(label_file,delimiter=',')

countO = 0
ms = []
for idx in range(500):
    at, props = qm9data.get_properties(idx)
    
    atom_numb = props['_atomic_numbers']
    number_atoms = len(props['_atomic_numbers'])
    
    for i in range(number_atoms):
        if atom_numb[i] == 8:
            if label['Target'][countO] == 2:
                at, props = qm9data.get_properties(idx)
                utils.write_xyz_from_db(props,idx)
                utils.xyz_to_mol(idx)
                
                molfile = './%d.mol' % (idx)
                mol = Chem.MolFromMolFile(molfile)
                ms.append(mol)
            countO = countO + 1  


In [6]:

    
fps = []    
#fps = [FingerprintMols.FingerprintMol(x) for x in ms]
    
df_fps = DataFrame(ms, columns=['mol'])
df_fps.head
#print(df_fps)

for index, row in df_fps.iterrows():
    fps_cal = FingerprintMols.FingerprintMol(row['mol'], minPath=1, maxPath=7, fpSize=2048, bitsPerHash=2, useHs=True, tgtDensity=0.0, minSize=128)
    fps.append([index, fps_cal])

fps2 = DataFrame(fps, columns=['idx','molecule'])
fps2 = fps2[fps2.columns[1]]
fps2 = fps2.values.tolist()

sim=[]
for n in range(len(fps2)):
    s = DataStructs.BulkTanimotoSimilarity(fps2[n], fps2[:])
    sim.append(s)

array = sim[1]
print(sim)
savetxt('./file.csv',sim,delimiter=',')

[[1.0, 0.4, 0.4, 0.4, 0.2, 0.25, 0.0, 0.0, 0.18181818181818182, 0.18181818181818182, 0.18181818181818182, 0.25, 0.13333333333333333, 0.10526315789473684, 0.10526315789473684, 0.13333333333333333, 0.13333333333333333, 0.16666666666666666, 0.15384615384615385, 0.14285714285714285, 0.1111111111111111, 0.2222222222222222, 0.2222222222222222, 0.2222222222222222, 0.2222222222222222, 0.2222222222222222, 0.2222222222222222, 0.25, 0.3333333333333333, 0.2857142857142857, 0.1111111111111111, 0.11428571428571428, 0.1111111111111111, 0.13333333333333333, 0.15384615384615385, 0.13333333333333333, 0.1, 0.16, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.07547169811320754, 0.0975609756097561, 0.0, 0.08695652173913043, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0136986301369863, 0.0, 0.0, 0.0, 0.013888888888888888, 0.0, 0.18181818181818182, 0.11764705882352941, 0.11764705882352941, 0.14285714285714285, 0.15384615384615385, 0.14285714285714285, 0.11764705882352941, 0.13333333333333333,

In [7]:
os.system('del -r *mol')
os.system('del -r *xyz')

0